In [59]:
from model import load_model, greedy_predict, tokens_to_text
from hooks import HookPoint, register_decoder_hook
from data import generate_dataset_pairs
import torch
import sympy

device = "cpu" # NOTE: change to cuda if your GPU can handle it

In [60]:
model = load_model(device=device)

/home/morris/miniconda3/envs/symreg/lib/python3.9/site-packages/pytorch_lightning/utilities/migration/migration.py:208: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../res/100m.ckpt`


In [61]:
# If we uncomment the code below, we set all decoder MLP outputs to random values using interventions.
#  As you'll see below, the model won't be able to fit the correct equation (:omg:).

"""
def test_hook(output, _hook: HookPoint):
    return torch.randn_like(output)

for layer in range(4):
    register_decoder_hook(model.model, test_hook, HookPoint(layer, "mlp"))
"""


'\ndef test_hook(output, _hook: HookPoint):\n    return torch.randn_like(output)\n\nfor layer in range(4):\n    register_decoder_hook(model.model, test_hook, HookPoint(layer, "mlp"))\n'

In [62]:
complexity_dataset = generate_dataset_pairs("complexity-bias", 500, 10, model.model_cfg, model.eq_cfg, second_dataset_sample_rate=2)

X = complexity_dataset["X0"][0]
y = complexity_dataset["y0"][0]

print("Ground truth function")
sympy.sympify(complexity_dataset["equations"][0][0])

Ground truth function


x_1*x_2*x_3 + x_1*x_2 + x_1

In [63]:
# initial token prediction, this initializes the sequence and caches the encoder embedding (saves computation time).
tok, seq, enc_embed = greedy_predict(model.model, model.params_fit, X.unsqueeze(0), y.unsqueeze(0))

# repeatedly predict next token greedily
for i in range(30):
    seq = greedy_predict(model.model, model.params_fit, enc_embed=enc_embed, sequence=seq)[1]

# this should result in (roughly) the correct equation
greedy_pred = tokens_to_text(seq, model.params_fit)

print("Greedy predicted equation:")
for eq in greedy_pred:
    display(sympy.sympify(eq))

/home/morris/miniconda3/envs/symreg/lib/python3.9/site-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Greedy predicted equation:


x_1*(x_2*x_3 + x_2) + x_1

In [64]:
# fit model with beam search instead of greedy + constant fitting (takes a lot longer)
output = model.fitfunc(X, y) 
# here you can see the fitted equations
output

Memory footprint of the encoder: 4.096e-05GB 



/home/morris/Documents/uni/msc/expl/NeuralSymbolicRegressionThatScales/src/nesymres/architectures/model.py:136: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X,device=self.device).unsqueeze(0)
/home/morris/Documents/uni/msc/expl/NeuralSymbolicRegressionThatScales/src/nesymres/architectures/model.py:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y,device=self.device).unsqueeze(0)


Constructing BFGS loss...
Flag idx remove ON, Removing indeces with high values...
checking input values range...
Attention, input values are very large. Optimization may fail due to numerical issues
Loss constructed, starting new BFGS optmization...
Constructing BFGS loss...
Flag idx remove ON, Removing indeces with high values...
checking input values range...
Attention, input values are very large. Optimization may fail due to numerical issues
Loss constructed, starting new BFGS optmization...


{'all_bfgs_preds': ['((x_1)+((x_1)*((x_2)+((x_2)*(x_3)))))',
  '((x_1)+((x_2)*((x_1)+((x_1)*(x_3)))))'],
 'all_bfgs_loss': [1.7166933e-10, 1.0823885e-10],
 'best_bfgs_preds': ['((x_1)+((x_2)*((x_1)+((x_1)*(x_3)))))'],
 'best_bfgs_loss': [1.0823885e-10]}

In [65]:
print("Best BFGS prediction:")
sympy.sympify(output["best_bfgs_preds"][0])

Best BFGS prediction:


x_1 + x_2*(x_1*x_3 + x_1)